In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
import time
from torch.cuda.amp import GradScaler, autocast


In [9]:
# Parameters
batch_size = 64
num_classes = 50
num_epochs = 10
learning_rate = 0.001
validation_split = 0.2

# Data transformations
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
# Load the dataset
dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/JPEGImages', transform=transformations)

# Splitting the dataset into train and validation sets
total_count = len(dataset)
val_count = int(total_count * validation_split)
train_count = total_count - val_count
train_dataset, val_dataset = random_split(dataset, [train_count, val_count])

# Create data loaders with increased num_workers and pin_memory
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

In [11]:
# Model setup
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scaler = GradScaler()

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# Training function
def train_model():
    start_time = time.time()  # Start time
    model.train()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i + 1) % 20 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        scheduler.step()
        print(f'Epoch [{epoch+1}/{num_epochs}] completed')

    end_time = time.time()  # End time
    total_time = end_time - start_time
    print(f'Total training time is {total_time:.2f} seconds')

In [13]:
# Evaluation function
def evaluate_model():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy of the model on the validation images: {100 * correct / total}%')

In [14]:
# Train and evaluate
train_model()
evaluate_model()

Epoch [1/10], Step [20/129], Loss: 2.1000
Epoch [1/10], Step [40/129], Loss: 2.1605
Epoch [1/10], Step [60/129], Loss: 2.5504
Epoch [1/10], Step [80/129], Loss: 1.8163
Epoch [1/10], Step [100/129], Loss: 1.7693
Epoch [1/10], Step [120/129], Loss: 1.5738
Epoch [1/10] completed
Epoch [2/10], Step [20/129], Loss: 1.2450
Epoch [2/10], Step [40/129], Loss: 1.4599
Epoch [2/10], Step [60/129], Loss: 1.2799
Epoch [2/10], Step [80/129], Loss: 1.3678
Epoch [2/10], Step [100/129], Loss: 1.2657
Epoch [2/10], Step [120/129], Loss: 1.3743
Epoch [2/10] completed
Epoch [3/10], Step [20/129], Loss: 0.9245
Epoch [3/10], Step [40/129], Loss: 0.9284
Epoch [3/10], Step [60/129], Loss: 1.2272
Epoch [3/10], Step [80/129], Loss: 0.9267
Epoch [3/10], Step [100/129], Loss: 1.1007
Epoch [3/10], Step [120/129], Loss: 1.2686
Epoch [3/10] completed
Epoch [4/10], Step [20/129], Loss: 0.9751
Epoch [4/10], Step [40/129], Loss: 0.6399
Epoch [4/10], Step [60/129], Loss: 0.6662
Epoch [4/10], Step [80/129], Loss: 0.8283
E